In [ ]:
# import pandas as pd

In [ ]:
# df = pd.read_csv(r'iCook_ingredient_res_analyze_used_v5_8_left_tag_v1.csv', engine='python', encoding='utf-8')
# df

In [ ]:
# len(df)

In [ ]:
# df.info()

In [ ]:
# mask9 = df["status_tag"] == 9
# df[mask9]

In [ ]:
# # 轉檔目標範例
# {
#   "recipe_name": "草莓鐵鍋鬆餅 Dutch baby",
#   "recipe_photo": "https://tokyo-kitchen.icook.network/uploads/recipe/cover/238037/large_620272afa3d9d4c8.jpg",
#   "materials": [
#     {
#       "material_name": "7吋鐵鍋(或烤盤)",
#       "material_amount": "1個"
#     },
#     {
#       "material_name": "雞蛋",
#       "material_amount": "1個份"
#     },
#     {
#       "material_name": "糖",
#       "material_amount": "6g"
#     }
#   ],
#   "recipe_url": "https://icook.tw/recipes/238037",
#   "recipe_category": "下午茶", # 這個欄位沒保留，改存下面的模型貼標
#   "model_tag": "$status_tag" # 原本資料的status_tag是模型的分類
# }

In [92]:
import csv
path = "res/iCook_ingredient_res_analyze_used_v5_8_left_tag_v1.csv"
with open(path, "r", encoding="utf-8") as infile:
    csvfile = csv.reader(infile)
    recipes_list = [d for d in csvfile]

In [ ]:
recipes_list

In [94]:
# tags不會用到，拿來建欄位參考用
tags = recipes_list[0]
tags

['status_tag', 'Cuisine', 'Ingredients', 'image', 'Url']

In [95]:
# 把tags欄位刪掉
recipes_list.remove(recipes_list[0])
recipes_list

In [47]:
# recipes_list[0][2].strip().split(";")

['pepper', 'water', 'chicken', 'salt', 'oil']

In [48]:
# 整理成存進 mongo 的 json 格式
# 版本一 materials = [] 不需存材料使用量的話，可以直接用 array 存起來
recipes_json_list = []
for recipe in recipes_list:
    status_tag = recipe[0]
    recipe_name = recipe[1]
    materials = recipe[2].strip().split(";")
    recipe_photo = recipe[3]
    recipe_url = recipe[4]
    
    recipe_json = {
        "recipe_name":recipe_name, 
        "recipe_photo":recipe_photo, 
        "materials": materials,
        "recipe_url":recipe_url,
        "model_tag":status_tag
    }
    
    recipes_json_list.append(recipe_json)

In [ ]:
recipes_json_list

In [55]:
from pymongo import MongoClient, errors
conn = MongoClient(host="localhost", port=27017)
db = conn.recipes
collection = db.tagged_recipes

inserted_id_list = []
for recipe in recipes_json_list:          
    try:
        recipe_id = collection.insert_one(recipe).inserted_id
        inserted_id_list.append(recipe_id)
    except errors.InvalidDocument as inv_doc_err:
        print(inv_doc_err)

In [4]:
from tools.countTimeDecorator import countTimeDecorator
from pymongo import MongoClient, errors

@countTimeDecorator
def recipesMongoQuery(db="recipes", collection="tagged_recipes", host="localhost", port=27017, query={}):
    conn = MongoClient(host=host, port=port)
    db = conn[db]
    collection = db[collection]
    result = collection.find(query)
    data = [d for d in result]
    return data

In [42]:
# Query 測試
"""
# official document
db.collection.find(  # collection
    { age: {$gte: 18}},  # query criteria
    { name: 1, address:1}  # projection (decide the fields to show)
).limit(5)  # cursor modifier

"""


query1 = {}
query2 = {"model_tag":"9"} # 把 model_tag = "9" 的食譜拉出來
query3 = {"materials":{"$in":["pork"]}} # 材料含有 pork
query4 = {"materials":{"$in":["pork","egg"]}} # 材料含有 pork 或 egg
query5 = {"$and":[{"materials":{"$in":["pork"]}},{"materials":{"$in":["egg"]}}]} #材料含有 pork 和 egg
query6 = {"materials":{"$size":2}}
query7 = {"materials":{"$not":{"$size":1}}}
query8 = {"$and":[{"materials":{"$not":{"$size":1}}},{"model_tag":"0"}]} # model_tag 是 "1" 且 materials含有兩種以上
query9 = {
    "model_tag":"0",
    "materials":{"$not":{"$size":0}},
    "materials":{"$not":{"$size":1}}
} # query8 = query9

In [43]:
# data1 = [d for d in collection.find(query1)]
# data2 = [d for d in collection.find(query2)]
# data3 = [d for d in collection.find(query3)]
# data4 = [d for d in collection.find(query4)]
# data5 = [d for d in collection.find(query5)]

data1 = recipesMongoQuery(query=query1)
data2 = recipesMongoQuery(query=query2)
data3 = recipesMongoQuery(query=query3)
data4 = recipesMongoQuery(query=query4)
data5 = recipesMongoQuery(query=query5)
data6 = recipesMongoQuery(query=query6)
data7 = recipesMongoQuery(query=query7)
data8 = recipesMongoQuery(query=query8)
data9 = recipesMongoQuery(query=query9)

recipesMongoQuery's time consumed(sec) is: 7.1399741702609585e-06
recipesMongoQuery's time consumed(sec) is: 3.1539466645982533e-06
recipesMongoQuery's time consumed(sec) is: 5.8194663789537216e-06
recipesMongoQuery's time consumed(sec) is: 5.4813755883110895e-06
recipesMongoQuery's time consumed(sec) is: 2.2594796286688913e-06
recipesMongoQuery's time consumed(sec) is: 1.8921825620863172e-06
recipesMongoQuery's time consumed(sec) is: 3.503296110365126e-06
recipesMongoQuery's time consumed(sec) is: 1.915891965230306e-06
recipesMongoQuery's time consumed(sec) is: 1.86357233259413e-06


In [44]:
print(len(data1))
print(len(data2))
print(len(data3))
print(len(data4))
print(len(data5))
print(len(data6))
print(len(data7))
print(len(data8))
print(len(data9))

861
71
61
511
30
63
798
36
36


In [ ]:
# 下面是檢查 conn = pymongo.MongoClient() 實例在指定 db 和 collection 後的參數變化

In [105]:
conn.__dict__

{'_MongoClient__options': <pymongo.client_options.ClientOptions at 0x114318128>,
 '_MongoClient__default_database_name': None,
 '_MongoClient__lock': <unlocked _thread.lock object at 0x118bbd6c0>,
 '_MongoClient__cursor_manager': None,
 '_MongoClient__kill_cursors_queue': [],
 '_event_listeners': <pymongo.monitoring._EventListeners at 0x114318278>,
 '_MongoClient__index_cache': {},
 '_MongoClient__index_cache_lock': <unlocked _thread.lock object at 0x118bbda30>,
 '_BaseObject__codec_options': CodecOptions(document_class=dict, tz_aware=False, uuid_representation=PYTHON_LEGACY, unicode_decode_error_handler='strict', tzinfo=None),
 '_BaseObject__read_preference': Primary(),
 '_BaseObject__write_concern': WriteConcern(),
 '_BaseObject__read_concern': ReadConcern(),
 '_MongoClient__all_credentials': {},
 '_topology_settings': <pymongo.settings.TopologySettings at 0x11431b0b8>,
 '_topology': <pymongo.topology.Topology at 0x11431b048>,
 '_kill_cursors_executor': <pymongo.periodic_executor.Per

In [108]:
conn[tdb].__dict__

{'_BaseObject__codec_options': CodecOptions(document_class=dict, tz_aware=False, uuid_representation=PYTHON_LEGACY, unicode_decode_error_handler='strict', tzinfo=None),
 '_BaseObject__read_preference': Primary(),
 '_BaseObject__write_concern': WriteConcern(),
 '_BaseObject__read_concern': ReadConcern(),
 '_Database__name': 'recipes',
 '_Database__client': MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True),
 '_Database__incoming_manipulators': [],
 '_Database__incoming_copying_manipulators': [],
 '_Database__outgoing_manipulators': [],
 '_Database__outgoing_copying_manipulators': []}

In [110]:
conn[tdb][tcoll].__dict__

{'_BaseObject__codec_options': CodecOptions(document_class=dict, tz_aware=False, uuid_representation=PYTHON_LEGACY, unicode_decode_error_handler='strict', tzinfo=None),
 '_BaseObject__read_preference': Primary(),
 '_BaseObject__write_concern': WriteConcern(),
 '_BaseObject__read_concern': ReadConcern(),
 '_Collection__database': Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'recipes'),
 '_Collection__name': 'tagged_recipes',
 '_Collection__full_name': 'recipes.tagged_recipes',
 '_Collection__write_response_codec_options': CodecOptions(document_class=dict, tz_aware=False, uuid_representation=PYTHON_LEGACY, unicode_decode_error_handler='replace', tzinfo=None)}